<a href="https://colab.research.google.com/github/atulshah16/EmergingTechnologiesCMPE297/blob/master/Assignment_2/Shah_Atul_014530243_CMPE_297_Assignment_2_part_c_Augmentation_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Atul Shah
Student ID: 014530243

# Augmentation Techniques using FastAI

In [2]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 3.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 358kB 23.5MB/s 
     |████████████████████████████████| 1.0MB 29.9MB/s 
     |████████████████████████████████| 40kB 6.8MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 40kB 7.5MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
     |████████████████████████████████| 2.6MB 44.3MB/s 
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *

## Training model using CIFAR10 dataset

In [4]:
from fastai.vision.all import *
path = untar_data(URLs.CIFAR)

In [5]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(3, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.356064,1.591986,0.466917,11:25
1,0.917505,0.918472,0.674583,11:28
2,0.710685,0.686783,0.758250,11:28


## Normalization

In [6]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([0.4632, 0.4563, 0.4221], device='cuda:0'),
 TensorImage([0.2272, 0.2220, 0.2357], device='cuda:0'))

In [7]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

In [8]:
dls = get_dls(64, 224)

In [9]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.0011,  0.0805,  0.1154], device='cuda:0'),
 TensorImage([1.1191, 1.0897, 1.1996], device='cuda:0'))

In [11]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(3, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.393984,1.590701,0.446750,11:21
1,0.908354,0.838401,0.703750,11:24
2,0.716400,0.681479,0.761750,11:25


## Progressive Resizing

In [11]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit_one_cycle(3, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.345580,1.396735,0.530750,05:00
1,0.900693,0.888983,0.678083,05:10
2,0.663457,0.618679,0.780333,05:10


In [12]:
learn.dls = get_dls(64, 224)
learn.fine_tune(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.844434,0.957653,0.682333,11:37


epoch,train_loss,valid_loss,accuracy,time
0,0.640630,0.654143,0.773667,11:49
1,0.537381,0.486899,0.835083,11:49
2,0.437490,0.413423,0.857083,11:49


## Test Time Augmentation

In [13]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

0.8640833497047424

## Mixup

In [15]:
#hide_input
#id mixup_example
#caption Mixing a church and a gas station
#alt An image of a church, a gas station and the two mixed up.
church = PILImage.create(get_image_files_sorted(path/'train'/'n03028079')[0])
gas = PILImage.create(get_image_files_sorted(path/'train'/'n03425413')[0])
church = church.resize((256,256))
gas = gas.resize((256,256))
tchurch = tensor(church).float() / 255.
tgas = tensor(gas).float() / 255.

_,axs = plt.subplots(1, 3, figsize=(12,4))
show_image(tchurch, ax=axs[0]);
show_image(tgas, ax=axs[1]);
show_image((0.3*tchurch + 0.7*tgas), ax=axs[2]);

IndexError: ignored

## Label Smoothing

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), 
                metrics=accuracy)
learn.fit_one_cycle(5, 3e-